**Live Demo of Object Detection using Google Colab Webcam**


# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Change Directory

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/objectdetection

/content/drive/MyDrive/Colab Notebooks/objectdetection


# Install Requirements

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 582 kB 74.2 MB/s 
     |████████████████████████████████| 408 kB 85.3 MB/s 
     |████████████████████████████████| 181 kB 68.7 MB/s 
     |████████████████████████████████| 144 kB 60.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 7.9 MB 15.8 MB/s 
     |████████████████████████████████| 596 kB 86.3 MB/s 
     |████████████████████████████████| 136 kB 89.5 MB/s 
     |████████████████████████████████| 1.1 MB 75.3 MB/s 
     |████████████████████████████████| 144 kB 78.6 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 271 kB 98.2 MB/s 
  Created wheel for Image: filename=image-1.5.33-py2.py3-none-any.whl size=19496 sha256=daf16c65855ff70187fa5cc5803ea71aeb1a27aab6d65e7724472c8310670701
  Stored in directory: /root/.cache/pip/wheels/56/88/e6/897194cfe8c08a8b9afd881d3bf53d102e13fa3960

# Program

In [10]:
#import functions 
import os
import pathlib
import cv2
import PIL
import matplotlib
import matplotlib.pyplot as plt
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import tensorflow_hub as hub
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
import html
import time
import torch
import argparse
#import detect_utils
import torchvision.transforms as transforms
import cv2
import numpy as np
import labelutils
import matplotlib.pyplot as plt
import utils
import torchvision
import sys
import math
import time
import datetime
import os
from LitDrinksModel import LitDrinksModel
import torchvision
import cv2
#import detect_utils
import config
import label_utils
from google.colab.patches import cv2_imshow


In [11]:
# define the torchvision image transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])



cur_dir = os.getcwd()
coco_names = config.params['classes']




# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array) # numpy array into PIL image object
  iobuf = io.BytesIO()
  
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')

  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes



  

def draw_boxes(boxes, classes, labels, image,scores):
    # read the image with OpenCV
    #image = np.array(image)
    
    class_names = classes
    rects = boxes   
    #print(class_names)
    font = cv2.FONT_HERSHEY_DUPLEX
    pos = (10,30)
    font_scale = 0.9
    font_color = (0, 0, 0)
    line_type = 1
    items = {}
    for i in range(len(scores)):
        rect = rects[i]
        x1 = rect[0]
        y1 = rect[1]
        x2 = rect[2]
        y2 = rect[3]
        #x2 = x1 + rect[2]
        #y2 = y1 + rect[3]
        x1 = int(x1)
        x2 = int(x2)
        y1 = int(y1)
        y2 = int(y2)
        name = class_names[i].split(":")[0]
        if name in items.keys():
            items[name] += 1
        else:
            items[name] = 1
        index = label_utils.class2index(name)
        color = label_utils.get_box_rgbcolor(index)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 3)
        # print(x1, y1, x2, y2, class_names[i])
        cv2.putText(image,
                    name,
                    (x1, y1-15),
                    font,
                    0.5,
                    color,
                    line_type)
    

    count = len(items.keys())
    if count > 0:
        xmin = 10
        ymin = 10
        xmax = 220
        ymax = 40 + count * 30
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 255, 255), thickness=-1)

        prices = config.params['prices']
        total = 0.0
        for key in items.keys():
            count = items[key]
            cost = count * prices[label_utils.class2index(key)]
            total += cost
            display = "%0.2f :%dx %s" % (cost, count, key)
            cv2.putText(image,
                        display,
                        (xmin + 10, ymin + 25),
                        font,
                        0.55,
                        (0, 0, 0),
                        1)
            ymin += 30

        cv2.line(image, (xmin + 10, ymin), (xmax - 10, ymin), (0,0,0), 1)

        display = "P%0.2f Total" % (total)
        cv2.putText(image,
                    display,
                    (xmin + 5, ymin + 25),
                    font,
                    0.75,
                    (0, 0, 0),
                    1)

    #cv2_imshow( image)

    return image


import itertools

def run_inference_for_single_image(model, image, live_cam,device,detection_threshold):
    
    # convert image into numpy
    image = np.asarray(image)
    #print('Converted image into numpy type:', type(image))
    
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    #input_tensor = tf.convert_to_tensor(image)
    input_tensor = transform(image).to(device)
    #print('Converted numpy into tensor format:', input_tensor)
    
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]
    
    # Run inference
    if not live_cam:
      start_time = time.time()
      outputs = model(input_tensor)
      end_time = time.time()
      print(f"Inference time: {np.ceil(end_time-start_time)} seconds per frame")

    #start_time = time.time()
    outputs = model(input_tensor)
    #end_time = time.time()
    #print(start_time-end_time)
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    pred_scores = pred_scores[pred_scores >= detection_threshold]
    num_detections = len(pred_scores)
    #outputs = dict(itertools.islice(outputs.items(), num_detections))
    outputs[0]['labels'] = outputs[0]['labels'][0:num_detections]
    pred_classes = [coco_names[i] for i in outputs[0]['labels'].cpu().numpy()]
    # get score for all the predicted objects
    #pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    # get all the predicted bounding boxes
    outputs[0]['boxes'] = outputs[0]['boxes'][0:num_detections]
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
    # get boxes above the threshold score
    #print(pred_scores)
    boxes = pred_bboxes[pred_scores >= detection_threshold].astype(np.int32)
    pred_scores = pred_scores[pred_scores >= detection_threshold]
    

    return boxes, pred_classes, outputs[0]['labels'] , pred_scores
        



    #output_dict = model(input_tensor)
    #num_detections = int(output_dict.pop('num_detections')) # 300

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    
    #output_dict = {key: value[0, :num_detections].numpy()
     #              for key, value in output_dict.items()}
    
    #output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    #output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    

    #return output_dict

def run_inference_video(model, video_path, live_cam):
  cap = cv2.VideoCapture(video_path)
  if cap.isOpened():
      width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
      height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
      res=(int(width), int(height))

      # save detected video
      # Initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*'mp4v') #codec
      curpath =os.getcwd()
      out = cv2.VideoWriter(curpath+'detected_output.mp4', fourcc, 30.0, res)
      frame = None
      device = 'cuda'
      detection_threshold = 0.7
      while True:
          try:
              is_success, image_np = cap.read()
          except cv2.error:
              continue

          if not is_success:
              break

          # Actual detection.
          with torch.no_grad():
           boxes, classes, labels ,scores = run_inference_for_single_image(model, image_np, live_cam,device,detection_threshold)
          
          # Visualization of the results of a detection.
          # draw boxes and show current frame on screen
          image =draw_boxes(boxes, classes, labels, frame ,scores)
               
          out.write(image)

      out.release() 

      # OPTIONAL: show last image
      if frame:
        cv2_imshow(frame)

  cap.release()



#Load model
model = LitDrinksModel.load_from_checkpoint('./weights/mymodel.ckpt')
model.eval().to('cuda')
detection_threshold = 0.7



# Video Demo

In [12]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = None
count = 0 
device = 'cuda'
detection_threshold = 0.7
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

     # Convert OpenCV BGR format to RGB format 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Actual detection.
    with torch.no_grad():
      boxes, classes, labels ,scores = run_inference_for_single_image(model, img,True,device,detection_threshold)
      
      # Visualization of the results of a detection.
      # draw boxes and show current frame on screen
    image =draw_boxes(boxes, classes, labels, img ,scores)
        

    # Convert OpenCV BGR format to RGB format 
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert numpy image type to Base64 image byte string type
    # for JavaScript Video object 
    bbox_bytes = bbox_to_bytes(image)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>